In [10]:
# 教學影片 AutoGen Quickstart - 5. Teaching Agents to Use Pre-Written Programs
# 1. Functions Basics.mp4

In [19]:
import autogen
from app import list_datasets

# 20240330 不懂 gpt-3.5-turbo 會出現錯誤，gpt-3.5-turbo-1106、gpt-3.5-turbo-16k 就不會 ????
config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-3.5-turbo-1106"]
    }
)

llm_config = {
    "config_list" : config_list,
    "temperature" : 0.2,
    "timeout" : 300,
    "functions": [{
				"name": "list_datasets",
				"description": "print out the titles ans URLs of datasets list on the main dataset catalog page",
				"parameters": {
					"type": "object",
					"properties": {
						"url": {
							"type": "string",
							"description": "url of the page.",  # 重要，提供給 agent 的訊息，讓他們知道如何使用該功能
							}
						},
					"required": ["url"],
	},
	}]
}

In [20]:
# 修改 system_message，以便任務達成後盡快結束對話。
assistant = autogen.AssistantAgent(
    name="assistant",
    system_message="""You are a helpful assistant. Once the purpose is achieved, Add a new line with the content of "終止, TERMINATE" at the end.""",
    llm_config=llm_config
)

user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",  # NEVER, TERMINATE, ALWAYS
    max_consecutive_auto_reply=6,
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        "work_dir": "agent_work_dir",
        "use_docker": False,
    },
    llm_config = llm_config
)

In [21]:
# 接著要註冊函數，以下是註冊函數的方法之一，function_map，是一個字典，用於將函數名映射到相應的函數對象上。不懂就是不懂，啥天書?
user_proxy.register_function(
	function_map={
		"list_datasets": list_datasets
	}
)

In [22]:
user_proxy.initiate_chat(
    assistant, 
    message = """Run the 'list_datasets' function and tell me which datasets would be useful for someone who wants
                 to write an article about public health. And tell me the reason why pick it in traditional chinese for each picked dataset. 
                 'https://catalog.data.gov/dataset' is the url."""
)


user_proxy (to assistant):

Run the 'list_datasets' function and tell me which datasets would be useful for someone who wants
                 to write an article about public health. And tell me the reason why pick it in traditional chinese for each picked dataset. 
                 'https://catalog.data.gov/dataset' is the url.

--------------------------------------------------------------------------------
assistant (to user_proxy):

***** Suggested function call: list_datasets *****
Arguments: 
{"url":"https://catalog.data.gov/dataset"}
**************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION list_datasets...
user_proxy (to assistant):

***** Response from calling function (list_datasets) *****

Title: Electric Vehicle Population Data
Link: /dataset/electric-vehicle-population-data

Title: Crime Data from 2020 to Present
Link: /dataset/crime-data-from-2020-to-present

Title: FD

ChatResult(chat_id=None, chat_history=[{'content': "Run the 'list_datasets' function and tell me which datasets would be useful for someone who wants\n                 to write an article about public health. And tell me the reason why pick it in traditional chinese for each picked dataset. \n                 'https://catalog.data.gov/dataset' is the url.", 'role': 'assistant'}, {'function_call': {'arguments': '{"url":"https://catalog.data.gov/dataset"}', 'name': 'list_datasets'}, 'content': None, 'role': 'assistant'}, {'content': "\nTitle: Electric Vehicle Population Data\nLink: /dataset/electric-vehicle-population-data\n\nTitle: Crime Data from 2020 to Present\nLink: /dataset/crime-data-from-2020-to-present\n\nTitle: FDIC Failed Bank List\nLink: /dataset/fdic-failed-bank-list\n\nTitle: Dynamic Small Business Search (DSBS)\nLink: /dataset/dynamic-small-business-search-dsbs-4f0da\n\nTitle: Motor Vehicle Collisions - Crashes\nLink: /dataset/motor-vehicle-collisions-crashes\n\nTitle: Lot